In [1]:
import tensorflow as tf

C:\Users\yasha\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\yasha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\yasha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm  
import tensorflow.keras
import math
from scipy import ndimage

In [4]:
train_dir = "E:\Projects\HandwrittenEquationSolverCNN\data\extracted_images"

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

     
train_datagen = ImageDataGenerator(rescale=1./255,
    
    data_format='channels_first',
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','+','-','times','(',')'],
    class_mode="sparse",
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','+','-','times','(',')'],
    class_mode="sparse",
    subset='validation') 

Found 148219 images belonging to 15 classes.
Found 37047 images belonging to 15 classes.


In [6]:
from tensorflow import keras

In [7]:
keras.backend.set_image_data_format('channels_first')

In [8]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers

In [9]:
inputs = keras.Input(shape=(1, 28, 28),)

fork11 = layers.Conv2D(64, 5, 5,  activation="relu", padding='same')(inputs)
fork12 = layers.Conv2D(64, 5, 5, activation="relu", padding='same')(inputs)
merge1 = layers.concatenate([fork11, fork12], axis=1, name='merge1')
# concat_feat = concatenate([concat_feat, x], mode='concat', axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
maxpool1 = layers.MaxPool2D(strides=(2,2), padding='same')(merge1)

fork21 = layers.Conv2D(128, 4, 4, activation="relu", padding='same')(maxpool1)
fork22 = layers.Conv2D(128, 4, 4, activation="relu", padding='same')(maxpool1)
merge2 = layers.concatenate([fork21, fork22, ], axis=1, name='merge2')
maxpool2 = layers.MaxPool2D(strides=(2,2), padding='same')(merge2)

fork31 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork32 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork33 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork34 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork35 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
fork36 = layers.Conv2D(256, 3, 3, activation="relu", padding='same')(maxpool2)
merge3 = layers.concatenate([fork31, fork32, fork33, fork34, fork35, fork36, ], axis=1, name='merge3')
maxpool3 = layers.MaxPool2D(strides=(2,2), padding='same')(merge3)

dropout = layers.Dropout(0.5)(maxpool3)

flatten = layers.Flatten()(dropout)
outputs = layers.Dense(15, activation="softmax")(flatten)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 28, 28)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 6, 6)     1664        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 6, 6)     1664        input_1[0][0]                    
__________________________________________________________________________________________________
merge1 (Concatenate)            (None, 128, 6, 6)    0           conv2d[0][0]                     
                                                                 conv2d_1[0][0]               

In [10]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adadelta(learning_rate = 1.0, rho=0.95),
    metrics = ["accuracy"]
)

In [11]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=100,
                    validation_steps=100,
                    epochs = 15) 

Epoch 1/15
100/100 [==============================] - 104s 961ms/step - loss: 2.4179 - accuracy: 0.1690 - val_loss: 2.4478 - val_accuracy: 0.2545
Epoch 2/15
100/100 [==============================] - 98s 981ms/step - loss: 2.0726 - accuracy: 0.3195 - val_loss: 1.7471 - val_accuracy: 0.4660
Epoch 3/15
100/100 [==============================] - 102s 1s/step - loss: 1.6753 - accuracy: 0.4580 - val_loss: 1.3401 - val_accuracy: 0.5890
Epoch 4/15
100/100 [==============================] - 86s 866ms/step - loss: 1.3877 - accuracy: 0.5415 - val_loss: 1.1968 - val_accuracy: 0.5930
Epoch 5/15
100/100 [==============================] - 97s 974ms/step - loss: 1.1090 - accuracy: 0.6295 - val_loss: 1.0009 - val_accuracy: 0.6635
Epoch 6/15
100/100 [==============================] - 82s 824ms/step - loss: 1.0121 - accuracy: 0.6740 - val_loss: 0.8570 - val_accuracy: 0.7350
Epoch 7/15
100/100 [==============================] - 66s 668ms/step - loss: 0.8805 - accuracy: 0.7105 - val_loss: 0.7325 - val_acc

### Training accuracy : 83.80% and Validation accuracy : 85.80%

In [12]:
model.save('E:\Projects\HandwrittenEquationSolverCNN\DeepCNN_15EY_sy.h5')